_Alex Malz, David Mykytyn_

This is a sandbox for developing an unsupervised classifier of astronomical lightcurves.

## Ongoing Ideas

_Special thanks to David Hogg_

### Better metric

- make it chisquare of arclength
- arclength must be relative to reference to account for different noise levels, or arclength relative to sum of both originals, then inverse transformation for symmetry, relative to null of no-overlap

complete sine/cosine basis -- rotation into space will look like linear fit, penalize high amplitude on high frequency modes --> probability, prior on frequency or number of modes, that would be a posterior probability

linearly interpolate one and get probability of other under same model

minimize 2nd derivative/sharpness

try different noise levels

try crazy outlier points, try arclength over all droppings of single point

many bands -- share shift but stretch will differ, can make a prior

### Regularization

- always looking at 3 year segment centered on overlap so never penalized by too long, enforce that there is overlap
- prior to make product of stretches = 1

chunks?

trigger time? probably not provided

regularizing erros: probably happens in space of variance, should be sensitive to shift in y, stretch in y, just need ratio of initial and final y values and initial variances

### Clustering

check symmetry requirements of distance metrics for clustering, i.e. tSNE needs convexity and symmetry

hierarchical clustering by combining best fits, reverse merger tree "active learning", both way branching is logN so we can visually monitor as we add new things

need a way to split out again, add one new thing, check against everything elses

In [ ]:
from collections import namedtuple
import itertools
import random
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
%matplotlib inline
import corner

# Simulate some mock data

We may need to preprocess to keep it reasonable, constraints on delta/stretch

What kinds of LC shapes might we have to worry about physically?  For now, just one transient (Gaussian) and one variable (sinusoid).  

In [ ]:
def make_gauss(scale, loc=0., amp=1., const=0.):
    func = sps.norm(loc, scale)
    out = lambda x: amp * func.pdf(x) + const
    return out

def make_sine(period, phase=0., amp=1., const=0.):
    func = lambda x: amp * (np.sin(period * x + phase)) + const
    return func

The lightcurves are sampled irregularly/sparsely in x and have observational errors/noise on y

In [ ]:
def make_cadence(x, scatter):
    assert(np.all((x[1:]-x[:-1]) > scatter))
    jitter = (np.random.uniform(np.shape(x)) - 0.5) * scatter * 2.
    perturbed = x + jitter
    return perturbed

def noisify_obs(y, scatter):
    errs = scatter * np.ones_like(y)
    new_y = y + sps.norm(0., scatter).rvs(np.shape(y))
    return(new_y, errs)

Okay, ready to make some data!

In [ ]:
def_cadence = np.arange(0., 200., 5.)

gmodel = make_gauss(10., 100., 50., 1.)
gtimes = make_cadence(def_cadence, 0.5)
gphot, gerr = noisify_obs(gmodel(gtimes), 0.1)

smodel = make_sine(20., 0., 5., 5.)
stimes = make_cadence(def_cadence, 0.5)
sphot, serr = noisify_obs(smodel(stimes), 0.3)

Define a lightcurve object for convenience

In [ ]:
# for now, limit to 2D: amplitude/flux/magnitude/color vs. time
LC = namedtuple('LC', ('x', 'y'))

In [ ]:
glc = LC(gtimes, gphot)
slc = LC(stimes, sphot)

What does it look like?

In [ ]:
plt.errorbar(glc.x, glc.y, yerr=gerr, linestyle='None', marker='o', label='Transient')
plt.errorbar(slc.x, slc.y, yerr=serr, linestyle='None', marker='+', label='Variable')
plt.legend()

# The overall strategy

We want to test the hypothesis that two lightcurves are noisy/sparse/irregular observations of the same object, under some permitted (afine) transformations.  Then we want to do clustering in the space of goodness-of-fit/consistency measure and the parameters of those transformations to identify classes.

In [ ]:
def merge(lca, lcb):
#     minx, maxx = max(min(lca.x), min(lcb.x)), min(max(lca.x), max(lcb.x))
    new_x = np.concatenate((lca.x, lcb.x))
    new_y = np.concatenate((lca.y, lcb.y))
    order = np.argsort(new_x)
    ord_x = new_x[order]
    ord_y = new_y[order]
#     condition = np.where(np.logical_and(ord_x <= maxx, ord_x >= minx))
#     ord_x = ord_x[condition]
#     ord_y = ord_y[condition]
    return LC(ord_x, ord_y)

regularize in time

In [ ]:
# def regx(lca0, lcb0, lcc):
    

## Permitted transformations

* shiftx
* stretchx
* shifty
* stretchy
* (cross-talk between bands)


also adjust error bars

In [ ]:
#affine transformations: translation vector, scaling/shear matrix, haven't propagated this yet
aff = namedtuple('aff', ('t', 's'))

In [ ]:
def transform(lc, deltax, deltay, stretchx, stretchy):
    new_x = (stretchx * lc.x) + deltax
    new_y = (stretchy * lc.y) + deltay
    return LC(new_x, new_y)

# Reduce to summary statistics (consistency metric)

Contenders:

* periodogram -- identify periodicity and stochastic noise levels, still okay to initially divide transient from variable
* flux per time bins -- trends keeping bin size constant but changing bin ends, i.e. moving window
* abs/percent change in color and total flux/magnitude

find MAP/MLE of p(A = B | lc_A, lc_B)
marginalize over shift/stretch params

Regularization is going to be really hard!

connect the dots is taking an arc length

could use the gaussian error bars to get probability that new hypothesis point lies on original line?

Random ideas: gaussian process kernels based on training set, get probability that connect-the-dots is drawn from that distribution; but, we know training set will be biased/incomplete, so will have to be able to adapt kernel, use some kind of exporation of space of kernels (gradient descent, genetic algorithm, etc.)

Consider the arclength.  If the two lightcurves came from the same object, then their arclengths should be comparable to one another (if over the same range).  Merging the lightcurves should not significantly affect the total arclength.  We could also consider an area between curves

In [ ]:
def connect_the_dots(lc):
    x_difs = (lc.x[1:] - lc.x[:-1])
    y_difs = lc.y[1:] - lc.y[:-1]
    sol = np.sqrt(x_difs ** 2 + y_difs ** 2)
    return np.sum(sol)

Let's try doing an optimization to find the transformation parameters minimizing the arclength ratio.

In [ ]:
def find_max_prob(lca, lcb, ivals=(0., 0., 1., 1.)):
    
    origa = connect_the_dots(lca)
    origb = connect_the_dots(lcb)
#     xrangea = np.max(lca.x)-np.min(lca.x)
#     yrangea = np.max(lca.y)-np.min(lca.y)
#     xrangeb = np.max(lcb.x)-np.min(lcb.x)
#     yrangeb = np.max(lcb.y)-np.min(lcb.y)
#     print (xrangea,yrangea,xrangeb,yrangeb)

    
#     xdifmaxa = np.max(lca.x[1:]-lca.x[:-1])
#     xdifmina = np.min(lca.x[1:]-lca.x[:-1])    
#     ydifmaxa = np.max(np.abs(lca.y[1:]-lca.y[:-1]))
#     ydifmina = np.min(np.abs(lca.y[1:]-lca.y[:-1]))
    
#     print (xdifmaxa,xdifmina,ydifmaxa,ydifmina)
    
#     xdifmaxb = np.max(lcb.x[1:]-lcb.x[:-1])
#     xdifminb = np.min(lcb.x[1:]-lcb.x[:-1])    
#     ydifmaxb = np.max(np.abs(lcb.y[1:]-lcb.y[:-1]))
#     ydifminb = np.min(np.abs(lcb.y[1:]-lcb.y[:-1]))
    
    
#     print (xdifmaxb,xdifminb,ydifmaxb,ydifminb)



    def dxlim(params):
        return xrangea - np.abs(params[0])
    def sxlim_hi(params):
        return xrangea - params[2] * xdifmaxb
    def sxlim_lo(params):
        return params[2] * xrangeb - xdifmaxa
    def sylim_hi(params):
        return yrangea - params[3] * ydifmaxb
    def sylim_lo(params):
        return params[3] * yrangeb - ydifmaxa

    def slim(params):
        return params[2:]

    constraints = [dxlim, sxlim_hi, sxlim_lo, sylim_hi, sylim_lo, slim]
    constraints = [{'type':'ineq','fun':x} for x in constraints]
    debug = []
    def _helper(params):
        (deltax, deltay, stretchx, stretchy) = params
        lc = transform(lcb, deltax, deltay, stretchx, stretchy)
        new_len = connect_the_dots(lc)
        lc_both = merge(lca, lc) 
        length = connect_the_dots(lc_both)
        to_min = length
        return(to_min)
    
    res = spo.minimize(_helper, ivals, method='Nelder-Mead', options={'maxiter':10000})#,constraints=constraints)
#     print(res)
    tmp = transform(lcb, res.x[0], res.x[1], res.x[2], res.x[3])
    fin = merge(lca, tmp)
    debug = connect_the_dots(fin)
    return(res, debug)

Let's try just doing this with merging and shifting for now and test it when the lightcurves have the same class.

In [ ]:
gtimes2 = gtimes + 50. * np.ones_like(gtimes)#make_cadence(def_cadence, 0.5)
gphot2, gerr2 = gphot, gerr#noisify_obs(gmodel(gtimes2), 0.1)
glc2 = LC(gtimes2, gphot2)
# glc2 = transform(glc, 5., 0., 1., 1.)
plt.plot(glc.x, glc.y, label='original')
plt.plot(glc2.x, glc2.y, label='shifted')
plt.plot(merge(glc, glc2).x, merge(glc, glc2).y, label='merged')
plt.legend()
print((connect_the_dots(glc), connect_the_dots(glc2), connect_the_dots(merge(glc, glc2)), connect_the_dots(merge(glc, transform(glc2, -0.5, 0., 1., 1.)))))

### an alternative to the arclength. . .


In [ ]:
# import chippr

# # prior_sigma = 0.16
# # prior_var = np.eye(n_bins)
# # for b in range(n_bins):
# #     prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
# # l = 1.e-4
# # prior_var = prior_var+l*np.identity(n_bins)



# a = 1.#amplitude
# b = 5.#inverse wavelength
# c = 1.e-2#random fluctuations
# #     prior_var = np.eye(n_bins)
# #     for k in range(n_bins):
# #         # print(k)
# #         prior_var[k] = a * np.exp(-0.5 * b * (z_mids[k] - z_mids) ** 2)
# #     prior_var += c * np.eye(n_bins)

# #     prior_mean = log_nz_intp
# #     # print('prior dimensions: '+str((np.shape(prior_mean), np.shape(prior_var))))
# #     prior = mvn(prior_mean, prior_var)
# #     if params['prior_mean'] is 'sample':
# #         prior_mean = prior.sample_one()
# #         prior = mvn(prior_mean, prior_var)

# # a = 1.# / n_bins
# # b = 20.#1. / z_difs ** 2
# # c = 0.
# prior_var = np.eye(len(gtimes))
# for k in range(len(gtimes)):
#     prior_var[k] = a * np.exp(-0.5 * b * (gtimes[k] - gtimes) ** 2)
# prior_var += c * np.identity(len(gtimes))

# prior_mean = gphot
# prior = chippr.mvn(prior_mean, prior_var)
# prior_samples = prior.sample(7)

In [ ]:
# plt.errorbar(gtimes, gphot, color='k', yerr=gerr, linestyle='None', marker='o')
# for i in range(7):
#     plt.plot(gtimes, prior_samples[i])
# plt.show()

### try the fitting methods

In [ ]:
ans, debug = find_max_prob(glc, glc2)

Let's plot the null hypothesis lightcurve, the test hypothesis lightcurve, transformation of the test hypothesis lightcurve making it comparible with the null hypothesis lightcurve, and merger of best transformation of the test hypothesis lightcurve and the null hypothesis lightcurve.

In [ ]:
def plot_reconstruct(lca, lcb, params, truea='', trueb=''):
    (dx, dy, sx, sy) = params
#     print(params)
    fin = transform(lcb, dx, dy, sx, sy)
#     print(fin.x)
    plt.plot(lca.x, lca.y, label='reference'+truea)
    plt.plot(lcb.x, lcb.y, label='hypothetical'+trueb)
    plt.plot(merge(lca, fin).x, merge(lca, fin).y, label='merged')
    plt.plot(fin.x, fin.y, label='transformed'+trueb)
    plt.title(str(params))
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
plot_reconstruct(glc, glc2, ans.x)

# Do this many times!

Set up the simulation parameters

In [ ]:
num_obj = 10
cls_models = [make_gauss, make_sine]
cls_params = [{'scale': 10., 'loc': 100., 'amp': 50., 'const': 1.}, 
              {'period': 20., 'phase': 0., 'amp': 5., 'const': 5.}]
cls_wts = None # even split for now
num_cls = len(cls_models)
# will need a way to draw model params

def_cadence = np.arange(0., 200., 5.)
lcs = []
truth = np.random.choice(range(num_cls), num_obj, p=cls_wts)
ids, inds, cts = np.unique(truth, return_counts=True, return_inverse=True)
# print(ids, cts, inds)

Make some lightcurves and record which are of the same class.

In [ ]:
for i in range(num_obj):
    times = make_cadence(def_cadence, 0.5)
    model = cls_models[ids[inds[i]]](**cls_params[ids[inds[i]]])
    phot, err = noisify_obs(model(times), 0.1)
    lcs.append(LC(times, phot))
    
masks = np.zeros((num_cls, num_obj, num_obj))
for i in ids:
    which_ones = np.where(truth == i)[0]
#     print(which_ones)
    pairs = np.array(list(itertools.permutations(which_ones, 2))).T
#     print(pairs)
    masks[i, pairs[0], pairs[1]] += 1
    
# print(masks)

Let's loop over the optimization, comparing pairwise.  We won't worry about skipping duplication yet because we can use it as a null test of whether this is working.

In [ ]:
def mini_pipeline(all_lcs):
    how_many = len(all_lcs)
    indices = range(how_many)
    dump_difs = np.empty((how_many, how_many))
    dump_params = []
    
    for i in indices:
        one_set = []
        for j in indices:
            ans, fin_len = find_max_prob(all_lcs[i], all_lcs[j])
#             print(i, j, ans, fin_len)
            one_set.append(np.asarray(ans.x))
            dump_difs[i][j] = fin_len
        dump_params.append(one_set)
    dump_params = np.array(dump_params)
            
    return(dump_params, dump_difs)

In [ ]:
all_params, all_difs = mini_pipeline(lcs)

In [ ]:
for i in range(num_obj):
    for j in range(num_obj):
        print((i, j, all_difs[i][j]))
        plot_reconstruct(lcs[i], lcs[j], all_params[i][j], truea=str(truth[i]), trueb=str(truth[j]))

## Visualizations below

In [ ]:
# check for symmetry -- really thought these would be symmetric. . .
plt.matshow(np.sum(masks, axis=0))
layered = np.swapaxes(all_params, 0, -1)

deltafunc = lambda x: np.abs(x)
stretchfunc = lambda x: np.min(np.array([x, 1./x]).T, axis=-1)
funcs = [deltafunc, deltafunc, stretchfunc, stretchfunc]

for i in range(4):
    plt.matshow(funcs[i](layered[i]))
    plt.plot([0, num_obj-1], [0, num_obj-1], color='k')

Visually, this doesn't seem to be working well

# Cluster in the space of summary statistics

kdtree (and more)

We want to see if the stretch/shear parameters for a class are clustered.

In [ ]:
global_mask = np.zeros((num_obj, num_obj))
# for i in range(4):
#     global_mask = np.logical_or(global_mask, masks[i])
for i in range(num_cls):
    global_mask = np.logical_or(global_mask, masks[i])
    plt.hist((all_difs * masks[i]).flatten(), alpha=0.25, label=str(i))
plt.hist(all_difs[~global_mask[i]].flatten(), alpha=0.25, label='no match')
plt.legend()

This sort of makes sense because we expect pairs of (s, 1/s) for stretch s and (t, -t) for translation t.

In [ ]:
corner.corner(all_params.reshape(100, 4))

Ouch, they really aren't symmetric nor interpretable.  But this is a very small sample size. . . so let's do it a little better.

# Let's try another approach :-}

In [ ]:
def listerize(data, masks):
    datashape = np.shape(data)
    global_mask = np.ma.make_mask_none(np.shape(masks)[1:])
    layers = []
    for i in range(len(masks)):# per class
        one_mask = np.ma.make_mask(masks[i])
        layer = np.ma.array(data, mask=np.ma.logical_not(one_mask)[np.newaxis])#data * masks[i][np.newaxis]
        global_mask = np.ma.mask_or(global_mask, one_mask)
        layers.append(layer.compressed())
        
    global_mask = np.ma.make_mask(global_mask)
    others = np.ma.array(data, mask=global_mask[np.newaxis]).compressed()#data * ~global_mask[np.newaxis]
    return(layers, others)

In [ ]:
per_class, mismatch = listerize(all_difs, masks)

In [ ]:
def density_estimation(m1, m2):
    X, Y = np.mgrid[min(m1):max(m1):100j, min(m2):max(m2):100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = sps.gaussian_kde(values)                                                             
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

def mycorner(data, keys, colors, maps, lims=None, pre_densities=None, filename='plot.pdf'):
    ncol = len(keys)
    fig = plt.figure(figsize=(ncol*5, ncol*5))
    ax = [[fig.add_subplot(ncol, ncol, ncol * i + j + 1) for j in range(i+1)] for i in range(ncol)]
#     print(len(data), len(colors))
    for k in range(len(data)):
        datum = data[k]
        npoints = len(datum)
        for i in range(ncol):
            for j in range(i+1):
                if i == j:
#                     print(datum[keys[i]])
                    ax[i][j].hist(datum[i].data, histtype='step', linewidth=2, alpha=0.5, color=colors[k])
                    ax[i][j].set_xlabel(keys[i])
                else:
#                     if (npoints >= 1e4 or npoints <= 100):
                    ax[i][j].scatter(datum[i].data, datum[j].data, color=colors[k], alpha=0.5)
#                     else:
#                         if pre_densities is None:
#                             x, y, z = density_estimation(datum[keys[i]], datum[keys[j]])
#                         else:
#                             (x, y, z) = pre_densities[i][j]
#                         ax[i][j].contour(x, y, z, cmap=plt.get_cmap(maps[k]) , alpha=0.5)
                    ax[i][j].set_xlabel(keys[i])
                    ax[i][j].set_ylabel(keys[j])
#                     if lims is not None:
#                         ax[i][j].set_xlim(lims)
#                         ax[i][j].set_ylim(lims)
#     fig.savefig(filename, dpi=100)
    return#(fig)
# replace with 2d histogram for speed

In [ ]:
print per_class[0].shape, per_class[1].shape
print len(mismatch)

In [ ]:
mycorner([per_class[0], per_class[1], mismatch], ['deltax', 'deltay', 'stretchx', 'stretchy'], ['r', 'g', 'b'], ['Reds', 'Greens', 'Blues'])

Nope, this still doesn't look like anything. . .

In [ ]:
for i in range(num_obj):
    for j in range(num_obj):
        plot_reconstruct(lcs[i], lcs[j], all_params[i][j], truea=str(truth[i]), trueb=str(truth[j]))

# Other ideas

pairwise combinations/comparisons?

space partitioning -- try to estimate

$$\int_{\theta \in D} p(x|\theta) d\theta$$

then we can iteratively refine. maybe have coarse upper/lower bounds by sampling discrete $\theta$ and multiplying by the volume of $D$, idk.

if we have independent observations $y_i$ and times $x_i$, maybe we want the product probability that some underlying function $f$ generated those points

$$\prod_i p(y_i | \theta)$$

if the error distributions are just gaussians with standard deviation $\sigma$ then the logprob is

$$\log \left(\prod_i \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{f(x_i)-y_i}{\sigma}\right)^2} \right)$$

which is for some constant $|C|$,

$$-|C| \sum_i \left(f(x_i) - y_i\right)^2$$

if $f$ is differentiable like the sine function, i think we can get a closed-form solution for finding the min (closest parameters). we probably want something more bayesian, but i imagine the probabilities fall off kinda fast from the min point, so having it as a reference might be nice.

In [ ]:
from scipy.stats import norm as scipy_norm

x = np.linspace(scipy_norm.ppf(0.01),
                scipy_norm.ppf(0.99), 100)
plt.plot(x, scipy_norm.pdf(x, 1, 2),
        'r-', lw=5, alpha=0.6, label='norm pdf')

In [ ]:
x = np.linspace(scipy_norm.ppf(0.01),
                scipy_norm.ppf(0.99), 100)
plt.plot(x, scipy_norm.pdf((x - 1) / 2),
        'r-', lw=5, alpha=0.6, label='norm pdf')

In [ ]:
def logsum_p_obs(expected, observed, noise_scale):
    diffs = 